<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-Vec-type" data-toc-modified-id="The-Vec-type-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The <code>Vec</code> type</a></span><ul class="toc-item"><li><span><a href="#Speedup-of-SIMD-vector-operations-vs-standard-arrays" data-toc-modified-id="Speedup-of-SIMD-vector-operations-vs-standard-arrays-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Speedup of SIMD vector operations vs standard arrays</a></span></li></ul></li><li><span><a href="#Operations-from-Vec-types-to-Vec-types" data-toc-modified-id="Operations-from-Vec-types-to-Vec-types-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Operations from <code>Vec</code> types to <code>Vec</code> types</a></span><ul class="toc-item"><li><span><a href="#Vector-operations" data-toc-modified-id="Vector-operations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Vector operations</a></span><ul class="toc-item"><li><span><a href="#Integer-operators:-[-+,--,-*,--&amp;,-|,-&lt;&lt;,-&gt;&gt;,-&gt;&gt;&gt;,-==,-!=,-&lt;,-&lt;=,-&gt;,-&gt;=]" data-toc-modified-id="Integer-operators:-[-+,--,-*,--&amp;,-|,-<<,->>,->>>,-==,-!=,-<,-<=,->,->=]-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Integer operators: <code>[ +, -, *,  &amp;, |, &lt;&lt;, &gt;&gt;, &gt;&gt;&gt;, ==, !=, &lt;, &lt;=, &gt;, &gt;=]</code></a></span></li><li><span><a href="#Float-operators:-[-+,--,-*,--&amp;,-|,-&lt;&lt;,-&gt;&gt;,-&gt;&gt;&gt;,-==,-!=,-&lt;,-&lt;=,-&gt;,-&gt;=]" data-toc-modified-id="Float-operators:-[-+,--,-*,--&amp;,-|,-<<,->>,->>>,-==,-!=,-<,-<=,->,->=]-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Float operators: <code>[ +, -, *,  &amp;, |, &lt;&lt;, &gt;&gt;, &gt;&gt;&gt;, ==, !=, &lt;, &lt;=, &gt;, &gt;=]</code></a></span></li></ul></li></ul></li><li><span><a href="#Elementwise-operation-in-Vec-elements" data-toc-modified-id="Elementwise-operation-in-Vec-elements-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Elementwise operation in <code>Vec</code> elements</a></span></li><li><span><a href="#Reduction-operations" data-toc-modified-id="Reduction-operations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Reduction operations</a></span><ul class="toc-item"><li><span><a href="#Shuffle" data-toc-modified-id="Shuffle-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Shuffle</a></span></li></ul></li><li><span><a href="#Performance-gain-by-lowering-precission" data-toc-modified-id="Performance-gain-by-lowering-precission-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Performance gain by lowering precission</a></span><ul class="toc-item"><li><span><a href="#Example-automatic-vectorization" data-toc-modified-id="Example-automatic-vectorization-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Example automatic vectorization</a></span></li><li><span><a href="#Exercise:-Algorithm-to-check-if-an-element-is-within-a-vector." data-toc-modified-id="Exercise:-Algorithm-to-check-if-an-element-is-within-a-vector.-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Exercise: Algorithm to check if an element is within a vector.</a></span></li><li><span><a href="#Exercise:-Algorithm-to-sum-over-indices-in-an-array" data-toc-modified-id="Exercise:-Algorithm-to-sum-over-indices-in-an-array-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Exercise: Algorithm to sum over indices in an array</a></span></li></ul></li></ul></div>

In [1]:
using SIMD
using BenchmarkTools

# The `Vec` type


SIMD vectors are similar to small fixed-size arrays of "simple" types.  These element types are supportedin SIMD.jl are:

```
- Bool 
- Int{8,16,32,64,128} 
- UInt{8,16,32,64,128} 
- Float{16,32,64}
```


We can create vector types (or `SIMD.Vec` types) as follows:

```
my_vec = Vec(a_tuple)
```

Notice that `eltype(a_tuple)` has to be in 

`[Bool, Int8, Int16, Int32, Int64, Int128, UInt8, UInt16, UInt32, UInt128, Float16, Float32, Float64]`

##### Examples:

```
a1_v = Vec(1,2,3,4,5,6,7,8)
a2_v = Vec(9,10,11,12,13,14,15,16,17,18)
```

##### Breaking examples:

```
a = Vec(("the","house")) # strings are not in the set of possible element types for a Vec
a = Vec((1,23.231))      # All elements in the tuple constructing the Vec need to be of the same type
```

We can operate with the given vectors as if they were arrays

In [2]:
a1_v = Vec((1,2,3,4,5,6,7,8))
a2_v = Vec((9,10,11,12,13,14,15,16))
res_v = a1_v + a2_v

<8 x Int64>[10, 12, 14, 16, 18, 20, 22, 24]

In [4]:
a1_a = [1,2,3,4,5,6,7,8]
a2_a = [9,10,11,12,13,14,15,16];
res_a = a1_a + a2_a

8-element Vector{Int64}:
 10
 12
 14
 16
 18
 20
 22
 24

Using operations between Vec types will be faster than the coutnerpart Array operations

In [8]:
print("Vector operation time: ")
println(@benchmark res_v = a1_v + a2_v)

print("Array operation time:  ")
println(@benchmark res_a = a1_a + a2_a)

Vector operation time: Trial(20.560 ns)
Array operation time:  Trial(51.692 ns)


## Speedup of SIMD vector operations vs standard arrays

In [10]:
a1_v = Vec(Int8.((1,2,3,4)))
a2_v = Vec(Int8.((1,2,3,4)))
print("Vector operation time: ")
@btime res = a1_v + a2_v

a1_a = [1,2,3,4]
a2_a = [1,2,3,4]
print("Array operation time:  ")
@btime res = a1_a + a2_a;

Vector operation time:   20.229 ns (1 allocation: 16 bytes)
Array operation time:    42.905 ns (1 allocation: 96 bytes)


In [11]:
a1_v = Vec(Int8.((1,2,3,4,5,6,7,8)))
a2_v = Vec(Int8.((1,2,3,4,5,6,7,8)))
print("Vector operation time: ")
@btime res = a1_v + a2_v;

a1_a = [1,2,3,4,5,6,7,8]
a2_a = [1,2,3,4,5,6,7,8]
print("Array operation time:  ")
@btime res = a1_a + a2_a;

Vector operation time:   19.690 ns (1 allocation: 16 bytes)
Array operation time:    46.040 ns (1 allocation: 128 bytes)


In [12]:
a1_v = Vec(Int64.(Tuple(x for x in 1:20)))
a2_v = Vec(Int64.(Tuple(x for x in 1:20)))
    
print("Vector operation time: ")
@btime res = a1_v + a2_v;

a1_a = [x for x in 1:20]
a2_a = [x for x in 1:20]
    
print("Array operation time: ")
@btime res = a1_a + a2_a;

Vector operation time:   30.034 ns (1 allocation: 272 bytes)
Array operation time:   53.946 ns (1 allocation: 224 bytes)


In [13]:
# error to be solved: If len is not even there can be some errors
# Discussion about this in https://github.com/eschnett/SIMD.jl/issues/61

#a1_v = Vec(Int64.((1,2,3,4,5,6,7)))
#a2_v = Vec(Int64.((1,2,3,4,5,6,7)))
#@btime res = a1_v + a2_v

The main difference of using standard vectors and Vec types is the fact that Vec types can be easily mapped to instructions that operate on the whole Vec. 


In [14]:
@code_native a1_v + a2_v

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.globl	"_julia_+_2159"                 ## -- Begin function julia_+_2159
	.p2align	4, 0x90
"_julia_+_2159":                        ## @"julia_+_2159"
; ┌ @ /Users/davidbuchaca/.julia/packages/SIMD/myoU9/src/simdvec.jl:252 within `+`
	.cfi_startproc
## %bb.0:                               ## %top
	movq	%rdi, %rax
; │ @ /Users/davidbuchaca/.julia/packages/SIMD/myoU9/src/simdvec.jl:253 within `+`
; │┌ @ /Users/davidbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:227 within `add`
; ││┌ @ /Users/davidbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:235 within `macro expansion`
	vmovdqu	128(%rdx), %ymm0
	vmovdqu64	(%rdx), %zmm1
	vmovdqu64	64(%rdx), %zmm2
	vpaddq	(%rsi), %zmm1, %zmm1
	vpaddq	64(%rsi), %zmm2, %zmm2
; │└└
	vpaddq	128(%rsi), %ymm0, %ymm0
	vmovdqa64	%zmm2, 64(%rdi)
	vmovdqa	%ymm0, 128(%rdi)
	vmovdqa64	%zmm1, (%rdi)
	vzeroupper
	retq
	.cfi_endproc
; └
                                

This does not happen in standard arrays (at least not as easily, the compiler could be smart enough to do it).

In [15]:
@code_native a1_a + a2_a

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.globl	"_julia_+_2195"                 ## -- Begin function julia_+_2195
	.p2align	4, 0x90
"_julia_+_2195":                        ## @"julia_+_2195"
; ┌ @ arraymath.jl:12 within `+`
	.cfi_startproc
## %bb.0:                               ## %top
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	andq	$-32, %rsp
	subq	$160, %rsp
	.cfi_offset %rbx, -56
	.cfi_offset %r12, -48
	.cfi_offset %r13, -40
	.cfi_offset %r14, -32
	.cfi_offset %r15, -24
	movq	%rsi, %r12
	movq	%rdi, %rbx
	movabsq	$4538966752, %r15               ## imm = 0x10E8B22E0
	vpxor	%xmm0, %xmm0, %xmm0
	vmovdqa	%ymm0, 32(%rsp)
	movabsq	$140733734880429, %rax          ## imm = 0x7FFF204684AD
	movl	$262, %edi                      ## imm = 0x106
	vzeroupper
	callq	*%rax
	movq	%rax, %r14
; │ @ arraymath.jl:14 within `+`
; │┌ @ indices.jl:1

	jmp	LBB0_39
LBB0_15:                                ## %L174.preheader
	cmpq	$16, %r13
	jae	LBB0_17
## %bb.16:
	xorl	%esi, %esi
	jmp	LBB0_22
LBB0_42:                                ## %vector.memcheck172
	leaq	(%rdx,%r13,8), %rsi
	leaq	8(%rcx), %rdi
	leaq	8(%rax), %r8
	cmpq	%rdi, %rdx
	setb	%bl
	cmpq	%rsi, %rcx
	setb	%r10b
	cmpq	%r8, %rdx
	setb	%dil
	cmpq	%rsi, %rax
	setb	%r8b
	xorl	%esi, %esi
	testb	%r10b, %bl
	jne	LBB0_47
## %bb.43:                              ## %vector.memcheck172
	andb	%r8b, %dil
	jne	LBB0_47
## %bb.44:                              ## %vector.ph191
	movq	%r13, %rsi
	andq	$-16, %rsi
	xorl	%edi, %edi
	.p2align	4, 0x90
LBB0_45:                                ## %vector.body189
                                        ## =>This Inner Loop Header: Depth=1
; ││││││ @ simdloop.jl:78 within `macro expansion`
; ││││││┌ @ int.jl:87 within `+`
	vmovq	(%rcx), %xmm0                   ## xmm0 = mem[0],zero
	vmovq	(%rax), %xmm1                   ## xmm1 = mem[0],zero
	vpaddq	%x

	addq	(%rcx), %rdi
; ││││││└└└└
; ││││││┌ @ array.jl:966 within `setindex!`
	movq	%rdi, (%rdx,%rsi,8)
; ││││││└
; ││││││ @ simdloop.jl:78 within `macro expansion`
; ││││││┌ @ int.jl:87 within `+`
	incq	%rsi
; ││││││└
; ││││││ @ simdloop.jl:75 within `macro expansion`
; ││││││┌ @ int.jl:83 within `<`
	cmpq	%rsi, %r13
; ││││││└
	jne	LBB0_39
	jmp	LBB0_48
LBB0_17:                                ## %vector.memcheck
	leaq	(%rdx,%r13,8), %rsi
	leaq	(%rcx,%r13,8), %rdi
	leaq	(%rax,%r13,8), %r8
	cmpq	%rdi, %rdx
	setb	%bl
	cmpq	%rsi, %rcx
	setb	%r10b
	cmpq	%r8, %rdx
	setb	%dil
	cmpq	%rsi, %rax
	setb	%r8b
	xorl	%esi, %esi
	testb	%r10b, %bl
	jne	LBB0_22
## %bb.18:                              ## %vector.memcheck
	andb	%r8b, %dil
	jne	LBB0_22
## %bb.19:                              ## %vector.ph
	movq	%r13, %rsi
	andq	$-16, %rsi
	xorl	%edi, %edi
	.p2align	4, 0x90
LBB0_20:                                ## %vector.body
                                        ## =>This Inner Loop Header: Depth=1
; ││

; │││││││┌ @ broadcast.jl:516 within `_bcs1`
; ││││││││┌ @ strings/io.jl:185 within `string`
	movabsq	$_ijl_box_int64, %r15
	vzeroupper
	callq	*%r15
	movq	%rax, %r14
	movq	%rax, 56(%rsp)
	movq	%rbx, %rdi
	callq	*%r15
	movq	%rax, 48(%rsp)
	movabsq	$4770284320, %rcx               ## imm = 0x11C54C320
	movq	%rcx, 64(%rsp)
	movq	%r14, 72(%rsp)
	movabsq	$4770284288, %rcx               ## imm = 0x11C54C300
	movq	%rcx, 80(%rsp)
	movq	%rax, 88(%rsp)
	movabsq	$_j1_print_to_string_2200, %rax
	movabsq	$4757498496, %rdi               ## imm = 0x11B91AA80
	leaq	64(%rsp), %rbx
	movq	%rbx, %rsi
	movl	$4, %edx
	callq	*%rax
	movq	%rax, 48(%rsp)
; ││││││││└
	movq	%rax, 64(%rsp)
	movabsq	$_ijl_apply_generic, %rax
	movabsq	$4758457600, %rdi               ## imm = 0x11BA04D00
	movq	%rbx, %rsi
	movl	$1, %edx
	callq	*%rax
	movabsq	$_ijl_throw, %rcx
	movq	%rax, %rdi
	callq	*%rcx
	.cfi_endproc
; └└└└└└└└
                                        ## -- End function
.subsections_via_symbols



# Operations from `Vec` types to `Vec` types


###### Operations between `Vec` elements


Let $Vec\{N,T\}$ be a set where a `Vec` can be instanciated. For example `Vec{8,Int32}`.
Here $N$ is the length of the vector and $T$ is the DataType. 


## Vector operations

There are essentially two types of vector operations.

###### Same data-type operators, where the input vectors have type $T$ and the output has type $T$

$$
Vec\{N,T\} \times Vec\{N,T\}  \longrightarrow Vec\{N,T\}
$$
Theese are `[+, -, *,  &, |, <<, >>, >>>,  / , ^]`


###### Comparation  operators, where the output has a boolean datatype

$$
Vec\{N,T\} \times Vec\{N,T\}  \longrightarrow Vec\{N,\text{Bool}\}
$$

Which are  `[==, !=, <, <=, >, >=]` and the output is a vector of booleans.



##### Float operations


Most operations between SIMD vector elements are functions of the form $Vec\{N,T \text{<:}\text{Float}\} \times Vec\{N,T\text{<:}\text{Float}\}  \longrightarrow Vec\{N,T \text{<:}\text{Float} \}$. We have the following methods in SIMD.jl:

```
+, -, *,  /,  ^
```


##### Integer operations

Some vector operations return a boolen vector  $Vec\{N,T\text{<:}\text{Int}\} \times Vec\{N,T\text{<:}\text{Int}\}  \longrightarrow Vec\{N,\text{<:}\text{Int}\}$. We have the following methods in SIMD.jl:


```
+ - * / % ^ !  & | $ << >> >>> 
```

##### Comparation operations

Some vector operations return a boolen vector  $Vec\{N,T\text{<:}\text{Int}\} \times Vec\{N,T\text{<:}\text{Int}\}  \longrightarrow Vec\{N,\text{Bool}\}$. We have the following methods in SIMD.jl:

```
== != < <= > >=
```


###### The "~" operator

There is an example of an operator that does not apply to two Vec elements. The "~" operator is $Vec\{N,T \text{<:}\text{Int}\}  \longrightarrow Vec\{N,T \text{<:} \text{Int}\}$.




###### Note: Vector operations cannot operate with vectors containing different data types

```julia
x = Vec(Int.((1,2,3,4)))
y = Vec(Float.((1,2,3,8)))
x + y

MethodError: no method matching +(::Vec{4,Int64}, ::Vec{4,Float64})
```

This is true even if both datatypes are subtypes of the same abstract type

```julia
x = Vec(Float32.((1,2,3,4)))
y = Vec(Float64.((1,2,3,8)))
x + y

MethodError: no method matching +(::Vec{4,Float32}, ::Vec{4,Float64})
```


In [52]:
x = Vec((1,2,6,4))
y = Vec((1,2,4,8))
x % y

<4 x Int64>[0, 0, 2, 4]

In [17]:
# Notice this does not work for Floats
# x = Vec((1.,2.,3.,4.))
# ~x

### Integer operators: `[ +, -, *,  &, |, <<, >>, >>>, ==, !=, <, <=, >, >=]`

In [53]:
x = Vec((1,2,3,4))
y = Vec((1,2,3,9))

# I should add to this list % operator
operations_integers = Symbol.([ +, -, *,  &, |, <<, >>, >>>, ==, !=, <, <=, >, >=])

for op in operations_integers
    println(op, "\tx $op y\t\t", eval(Meta.parse("x $(op) y"))) #, #eval(exp(x op y)))
end

+	x + y		<4 x Int64>[2, 4, 6, 13]
-	x - y		<4 x Int64>[0, 0, 0, -5]
*	x * y		<4 x Int64>[1, 4, 9, 36]
&	x & y		<4 x Int64>[1, 2, 3, 0]
|	x | y		<4 x Int64>[1, 2, 3, 13]
<<	x << y		<4 x Int64>[2, 8, 24, 2048]
>>	x >> y		<4 x Int64>[0, 0, 0, 0]
>>>	x >>> y		<4 x Int64>[0, 0, 0, 0]
==	x == y		<4 x Bool>[1, 1, 1, 0]
!=	x != y		<4 x Bool>[0, 0, 0, 1]
<	x < y		<4 x Bool>[0, 0, 0, 1]
<=	x <= y		<4 x Bool>[1, 1, 1, 1]
>	x > y		<4 x Bool>[0, 0, 0, 0]
>=	x >= y		<4 x Bool>[1, 1, 1, 0]


###### Be carefull with operations because they can behave in a wrap-around (or modulus) fashion


In the following cell we can see that 255+1 gives 0 as result and 255+2 gives 1 as result.

In [69]:
x = Vec(UInt8.((255,255,255,255)))
y = Vec(UInt8.((1,2,0,0)))
x + y

<4 x UInt8>[0x00, 0x01, 0xff, 0xff]

We can proactively avoid this using `add_saturate` operation.
This operation keeps the `typemax` of the elementype if the result exceedds it (instead of doing a wrap around )

In [71]:
# Put Here a ceil version
x = Vec(UInt8.((255,255,255,255)))
y = Vec(UInt8.((1,0,0,0)))
SIMD.add_saturate(x, y)

<4 x UInt8>[0xff, 0xff, 0xff, 0xff]

### Float operators: `[ +, -, *,  &, |, <<, >>, >>>, ==, !=, <, <=, >, >=]`

In [21]:
x = Vec((1.,2.,3.,4.))
y = Vec((1.,2.,3.,9.))

operations_floats = Symbol.([ +, -, *,  /,  ^,  ==, !=, <, <=, >, >=])

for op in operations_floats
    println(op, "\tx $op y\t\t", eval(Meta.parse("x $op y"))) #, #eval(exp(x op y)))
end

+	x + y		<4 x Float64>[2.0, 4.0, 6.0, 13.0]
-	x - y		<4 x Float64>[0.0, 0.0, 0.0, -5.0]
*	x * y		<4 x Float64>[1.0, 4.0, 9.0, 36.0]
/	x / y		<4 x Float64>[1.0, 1.0, 1.0, 0.4444444444444444]
^	x ^ y		<4 x Float64>[1.0, 4.0, 27.0, 262144.0]
==	x == y		<4 x Bool>[1, 1, 1, 0]
!=	x != y		<4 x Bool>[0, 0, 0, 1]
<	x < y		<4 x Bool>[0, 0, 0, 1]
<=	x <= y		<4 x Bool>[1, 1, 1, 1]
>	x > y		<4 x Bool>[0, 0, 0, 0]
>=	x >= y		<4 x Bool>[1, 1, 1, 0]



# Elementwise operation in `Vec` elements

An elementwise operation in a vec element is a function $Vec\{N,T\} \longrightarrow Vec\{N,T\}$

The following operations are available:

```
abs cbrt ceil copysign cos div exp exp10 exp2 flipsign floor fma inv isfinite isinf isnan issubnormal log log10 log2 muladd rem round sign signbit sin sqrt trunc vifelse
```

In [22]:
x = Vec((1.,2.,3.,4.))

<4 x Float64>[1.0, 2.0, 3.0, 4.0]

In [23]:
exp(x)

<4 x Float64>[2.718281828459045, 7.38905609893065, 20.085536923187668, 54.598150033144236]

In [24]:
ceil(x)

<4 x Float64>[1.0, 2.0, 3.0, 4.0]



# Reduction operations

A reduction operation is a function of the form $Vec\{N,T\}  \longrightarrow T$.

Therefore reductions "reduce" a SIMD vector to a scalar. The following reduction operations are provided:

```
all any maximum minimum sum prod
```

In [25]:
x = Vec((1.,2.,3.,4.))
sum(x)

10.0

In [26]:
x = Vec((1,2,3,4))
sum(x)

10

## Shuffle

`shufflevector` allows a mask to shuffle elements in a Vec

In [233]:
a = Vec{4, Int32}((1,2,3,4))
mask = (0,3,1,2)
shufflevector(a, Val(mask))

<4 x Int32>[1, 4, 2, 3]

Note that the mask can take the same 

In [245]:
a = Vec{4, Int32}((1,2,3,4))
mask = (0,0,1,2)
shufflevector(a, Val(mask))

<4 x Int32>[1, 1, 2, 3]

In [239]:
@benchmark sum(shufflevector(a, Val(mask)))

BenchmarkTools.Trial: 10000 samples with 221 evaluations.
 Range (min … max):  334.814 ns …  89.489 μs  ┊ GC (min … max): 0.00% … 99.37%
 Time  (median):     344.941 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   395.493 ns ± 894.921 ns  ┊ GC (mean ± σ):  2.25% ±  0.99%

  ▆█▅▃▁ ▁     ▁▁▁▁▂▂▂▁▂▁▁▁ ▁                                    ▁
  █████████████████████████████████████████▇▇▇▇▆▆▆▆▆▆▆▆▅▆▄▅▆▅▅▄ █
  335 ns        Histogram: log(frequency) by time        690 ns <

 Memory estimate: 80 bytes, allocs estimate: 2.

The same function can be used with two input vectors, and the mask is applied to positions of both vectors

In [226]:
                  #0,1,2,3
a = Vec{4, Int32}((1,2,3,4))
                  #4,5,6,7
b = Vec{4, Int32}((5,6,7,8))
mask = (2,3,4,5)
shufflevector(a, b, Val(mask))

<4 x Int32>[3, 4, 5, 6]

# Performance gain by lowering precission

The speed difference can be magnified if we do more operations than a single sum:

In [27]:
function sum_vec(a1, a2, b1, b2)
    res1 = a1 + a2
    res2 = b1 + b2
    return res1, res2
end

sum_vec (generic function with 1 method)

In [28]:
@btime sum_vec(a1_a, a2_a, a1_a, a2_a)

  107.177 ns (3 allocations: 480 bytes)


([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40])

In [29]:
@btime sum_vec(a1_v, a2_v, a1_v, a2_v)

  46.208 ns (1 allocation: 544 bytes)


(<20 x Int64>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], <20 x Int64>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40])

In [31]:
function sum_vec_4(a1,a2,b1,b2)
    res1 = a1 + a2
    res2 = b1 + b2
    res3 = a1 + b1
    res4 = a2 + b2

    return res1, res2, res3, res4
end

sum_vec_4 (generic function with 1 method)

In [32]:
@btime sum_vec_4(a1_v,a2_v,a1_v,a2_v)

  78.090 ns (1 allocation: 1.06 KiB)


(<20 x Int64>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], <20 x Int64>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], <20 x Int64>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], <20 x Int64>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40])

In [34]:
@btime sum_vec_4(a1_a, a2_a, a1_a, a2_a)

  170.673 ns (5 allocations: 944 bytes)


([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40])

Notice that the speed gain between the prevoius two calls is, at most, 4X.

We can increase the difference in speed using an integer type with less bits. For example Int32.

In [35]:
a1_i8_v = Vec(tuple(Int32.(a1_a)...))
a2_i8_v = Vec(tuple(Int32.(a2_a)...))

<20 x Int32>[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [36]:
@btime sum_vec_4(a1_i8_v,a2_i8_v,a1_i8_v,a2_i8_v)

  47.582 ns (1 allocation: 544 bytes)


(<20 x Int32>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], <20 x Int32>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], <20 x Int32>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40], <20 x Int32>[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40])

## Example automatic vectorization

In [37]:
function c_a_times_b!(c::Array, a::Array, b::Array)
    @assert length(a) == length(b) == length(c)
    @inbounds for i in 1:length(a)
        c[i] = a[i] * b[i]
    end
end

c_a_times_b! (generic function with 1 method)

In [38]:
 V64 = Vector{Float64}

Vector{Float64} (alias for Array{Float64, 1})

Inspecting `code_llvm` we can see

```
 ┌ @ float.jl:399 within `*'
   %55 = fmul <4 x double> %wide.load, %wide.load24
   %56 = fmul <4 x double> %wide.load21, %wide.load25
   %57 = fmul <4 x double> %wide.load22, %wide.load26
   %58 = fmul <4 x double> %wide.load23, %wide.load27
```

In [47]:
# Uncomment the following line and run
#code_llvm(c_a_times_b!, Tuple{V64, V64, V64})

Inspecting `code_native`

```
	vmovupd	(%r10,%rcx,8), %ymm0
	vmovupd	32(%r10,%rcx,8), %ymm1
	vmovupd	64(%r10,%rcx,8), %ymm2
	vmovupd	96(%r10,%rcx,8), %ymm3
```

In [46]:
# Uncomment the following line and run
#code_native(c_a_times_b!, Tuple{V64, V64, V64})

## Exercise: Algorithm to check if an element is within a vector.

In [77]:
function find_val_in_array(x, val)
        
    @inbounds for i in 1:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end

find_val_in_array (generic function with 1 method)

In [78]:
x = Array{Int32}(1:1_000_000);
va = Int32(500_000);
@btime find_val_in_array(x, va)

  151.972 μs (0 allocations: 0 bytes)


true

In [75]:

function find_val_in_array_simd(x, val)
    n_simd = 32
    last_pos_simd_chunk = length(x)-n_simd
    @inbounds for i in 1:n_simd:last_pos_simd_chunk
        vec_i = vload(Vec{n_simd, Int32}, x, i)
        sum_equality = sum(vec_i == val)
        if sum_equality >0
            return true
        end
    end

    @inbounds for i in last_pos_simd_chunk:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end

find_val_in_array_simd (generic function with 1 method)

In [76]:
x = Array{Int32}(1:1_000_000);
va = Int32(500_000);
@btime find_val_in_array_simd(x, va)

  30.508 μs (0 allocations: 0 bytes)


true

Note that we implemented this function hardcoding the elementtype `Int32` as well as the vector length `n_simd`.

We can avoid this.

In [79]:

function find_val_in_array_simd(x::Array{T}, val::T) where {T}
    n_simd = 64
    last_pos_simd_chunk = length(x)-n_simd
    @inbounds for i in 1:n_simd:last_pos_simd_chunk
        vec_i = vload(Vec{n_simd, T}, x, i)
        sum_equality = sum(vec_i == val)
        if sum_equality > 0
            return true
        end
    end

    @inbounds for i in last_pos_simd_chunk:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end

find_val_in_array_simd (generic function with 2 methods)

In [80]:
x = Array{Int32}(1:1_000_001);
va = Int32(500_000);
@btime  find_val_in_array_simd(x, va)

  30.447 μs (0 allocations: 0 bytes)


true

## Exercise: Algorithm to sum over indices in an array

In [96]:
using BenchmarkTools
using LoopVectorization

n_clusters = 32
n_examples = 1_000_000
n_features = 128

T = Float32.(rand(n_clusters, n_features));                   # ADC table
PQ = UInt8.(rand(1:n_clusters, n_features, n_examples))    # PQcodes
y = Float32.(rand(n_features));

function lsh(PQ, T)
    
    n_features, n_examples = size(PQ)
    d = zeros(eltype(T), n_examples)
    
    @inbounds for n in 1:n_examples
        res = zero(eltype(T))
        @fastmath  for j in 1:n_features
            res += T[PQ[j,n],j]    
        end
        d[n] = res
    end
    return d
end


lsh (generic function with 1 method)

In [97]:
@benchmark lsh($PQ, $T)

BenchmarkTools.Trial: 105 samples with 1 evaluation.
 Range (min … max):  40.448 ms … 73.327 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     47.359 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   47.673 ms ±  4.298 ms  ┊ GC (mean ± σ):  0.51% ± 2.96%

            ▁        █▄▄▂▁█▂                                   
  ▃▁▅▃▅▁▃▃▆▃█▅▃▃▆▅▅▃████████▃█▆▃▁▃▁▅▃▁▃▁▃▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃▁▁▁▁▆ ▃
  40.4 ms         Histogram: frequency by time        59.5 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

In [255]:
v = PQ[:,1]
pq_1 = PQ[:,1]
pq_1_vec = vload(Vec{128,UInt8}, v, 1)

<128 x UInt8>[0x01, 0x14, 0x0e, 0x09, 0x11, 0x1c, 0x09, 0x17, 0x02, 0x0f, 0x0e, 0x19, 0x17, 0x07, 0x1c, 0x04, 0x08, 0x03, 0x19, 0x04, 0x04, 0x06, 0x18, 0x01, 0x10, 0x16, 0x0b, 0x0a, 0x0f, 0x17, 0x19, 0x1a, 0x04, 0x0a, 0x0a, 0x0f, 0x1a, 0x0b, 0x07, 0x03, 0x1d, 0x01, 0x08, 0x14, 0x1c, 0x14, 0x02, 0x18, 0x0c, 0x18, 0x12, 0x18, 0x12, 0x13, 0x0b, 0x02, 0x0c, 0x01, 0x13, 0x0d, 0x14, 0x03, 0x15, 0x06, 0x04, 0x1e, 0x0f, 0x03, 0x05, 0x1e, 0x07, 0x1a, 0x02, 0x1f, 0x1e, 0x09, 0x13, 0x06, 0x01, 0x0f, 0x13, 0x10, 0x0f, 0x17, 0x1e, 0x13, 0x13, 0x1f, 0x20, 0x11, 0x11, 0x0b, 0x0c, 0x20, 0x12, 0x10, 0x13, 0x15, 0x1c, 0x07, 0x0b, 0x07, 0x11, 0x05, 0x12, 0x16, 0x11, 0x17, 0x0a, 0x12, 0x02, 0x20, 0x06, 0x1f, 0x1f, 0x0b, 0x11, 0x09, 0x09, 0x0b, 0x1b, 0x0d, 0x0b, 0x08, 0x09, 0x11, 0x08, 0x05]

In [256]:
@benchmark sum(pq_1_vec)

BenchmarkTools.Trial: 10000 samples with 998 evaluations.
 Range (min … max):  12.668 ns … 135.888 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.988 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.474 ns ±   4.001 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅▂ ▁█▅▂                                                      ▁
  ██▆████▇▆▇▇▇▇▆▆▅▆▆▅█▆▆▅▅▅▆▄▃▅▄▄▄▅▄▅▄▄▆▅▅▄▅▅▅▄▅▅▅▄▅▅▅▁▄▄▅▅▃▄▄ █
  12.7 ns       Histogram: log(frequency) by time      30.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [257]:
@benchmark sum(pq_1)

BenchmarkTools.Trial: 10000 samples with 993 evaluations.
 Range (min … max):  33.901 ns … 246.870 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     35.841 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   41.170 ns ±  12.795 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▆▅▃▁▁                  ▁▁▁▁▁▁▁▁                            ▁
  ██████████▇█▇▇▇▇▇▇███▇████████████▇▇▆▆▅▅▅▅▄▅▄▄▅▅▆▄▅▃▅▆▆▅▆▅▆▆ █
  33.9 ns       Histogram: log(frequency) by time      91.8 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

We could

In [267]:
table_vec = vload(Vec{32,Float32}, T[:,1], 1)

<32 x Float32>[0.95423836, 0.673387, 0.5049787, 0.48568603, 0.3124652, 0.65885574, 0.5401918, 0.623749, 0.40774092, 0.94546694, 0.47636735, 0.89755404, 0.5313211, 0.04663205, 0.61256987, 0.4160553, 0.9608643, 0.1554406, 0.9803495, 0.9772245, 0.35994515, 0.22976878, 0.5784012, 0.12941517, 0.33469325, 0.92864996, 0.6486976, 0.41019225, 0.90692693, 0.8520791, 0.87212634, 0.012455661]

In [279]:
#vload(Vec{32,Float32}, T[:,1], 1)
#pq_1_vec

In [283]:
#T[:,1]
vload(Vec{32,Float32}, T[:,1], 1)

<32 x Float32>[0.95423836, 0.673387, 0.5049787, 0.48568603, 0.3124652, 0.65885574, 0.5401918, 0.623749, 0.40774092, 0.94546694, 0.47636735, 0.89755404, 0.5313211, 0.04663205, 0.61256987, 0.4160553, 0.9608643, 0.1554406, 0.9803495, 0.9772245, 0.35994515, 0.22976878, 0.5784012, 0.12941517, 0.33469325, 0.92864996, 0.6486976, 0.41019225, 0.90692693, 0.8520791, 0.87212634, 0.012455661]

In [284]:
#vload(Vec{32,Float32}, T[:,1], 1)

In [314]:
idx = PQ[:,1];

In [302]:
Int(maximum(idx))

32

In [310]:
#vgather(T[:,1], idx)

In [313]:
# T[:,1]

In [176]:
x = T[:,1] ;

In [214]:
a1_v = Vec((1.,2.,3.,4.,5.,6.,7.,8.))
idx = Vec(Int.((3, 3, 3, 3, 3, 3, 3, 2)))
x_idx = vgather(x, idx)

<8 x Float32>[0.5049787, 0.5049787, 0.5049787, 0.5049787, 0.5049787, 0.5049787, 0.5049787, 0.673387]

In [211]:
#vgather(x, idx)

In [196]:
@benchmark @inbounds sum(vgather(x, idx))

LoadError: MethodError: no method matching vgather(::Vector{Float32}, ::Vec{8, UInt8})
[0mClosest candidates are:
[0m  vgather(::Union{Base.ReinterpretArray{T, 1, T2, A} where {T2, A<:Union{DenseVector{T2}, SubArray{T2, 1, P, I, true} where {P, I<:Union{Tuple{Vararg{Real}}, Tuple{AbstractUnitRange, Vararg{Any}}}}}}, SubArray{T, 1, P, I, true} where {P, I<:Union{Tuple{Vararg{Real}}, Tuple{AbstractUnitRange, Vararg{Any}}}}, DenseVector{T}}, [91m::Vec{N, Int64}[39m) where {N, T<:Union{Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8}} at ~/.julia/packages/SIMD/myoU9/src/arrayops.jl:157
[0m  vgather(::Union{Base.ReinterpretArray{T, 1, T2, A} where {T2, A<:Union{DenseVector{T2}, SubArray{T2, 1, P, I, true} where {P, I<:Union{Tuple{Vararg{Real}}, Tuple{AbstractUnitRange, Vararg{Any}}}}}}, SubArray{T, 1, P, I, true} where {P, I<:Union{Tuple{Vararg{Real}}, Tuple{AbstractUnitRange, Vararg{Any}}}}, DenseVector{T}}, [91m::Vec{N, Int64}[39m, [91m::Vec{N, Bool}[39

In [190]:
table_vec = vload(Vec{32,Float32}, T[:,1], 1)

<32 x Float32>[0.95423836, 0.673387, 0.5049787, 0.48568603, 0.3124652, 0.65885574, 0.5401918, 0.623749, 0.40774092, 0.94546694, 0.47636735, 0.89755404, 0.5313211, 0.04663205, 0.61256987, 0.4160553, 0.9608643, 0.1554406, 0.9803495, 0.9772245, 0.35994515, 0.22976878, 0.5784012, 0.12941517, 0.33469325, 0.92864996, 0.6486976, 0.41019225, 0.90692693, 0.8520791, 0.87212634, 0.012455661]

In [193]:
dd

<8 x Int64>[3, 3, 3, 3, 3, 3, 3, 2]

In [147]:
i = 1
arr = Vector{Float64}(undef, 100)
xs = vload(Vec{4,Float64}, arr, i)

<4 x Float64>[0.0, 0.0, 0.0, 0.0]

In [146]:
xs + xs

<4 x Float64>[0.0, 0.0, 0.0, 0.0]